In [13]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            
print('Training complete.')

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy: %d %%' %(100 * correct / total))


Files already downloaded and verified
Files already downloaded and verified
[1,  2000] loss: 2.215
[1,  4000] loss: 1.831
[1,  6000] loss: 1.694
[1,  8000] loss: 1.602
[1, 10000] loss: 1.537
[1, 12000] loss: 1.475
[2,  2000] loss: 1.413
[2,  4000] loss: 1.365
[2,  6000] loss: 1.352
[2,  8000] loss: 1.356
[2, 10000] loss: 1.273
[2, 12000] loss: 1.288
[3,  2000] loss: 1.211
[3,  4000] loss: 1.227
[3,  6000] loss: 1.179
[3,  8000] loss: 1.204
[3, 10000] loss: 1.212
[3, 12000] loss: 1.167
[4,  2000] loss: 1.114
[4,  4000] loss: 1.090
[4,  6000] loss: 1.113
[4,  8000] loss: 1.114
[4, 10000] loss: 1.116
[4, 12000] loss: 1.097
[5,  2000] loss: 1.033
[5,  4000] loss: 1.042
[5,  6000] loss: 1.057
[5,  8000] loss: 1.043
[5, 10000] loss: 1.053
[5, 12000] loss: 1.044
Training complete.
Accuracy: 60 %
